In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=8dcc5cbd4afc0b289b12b1d86068a0bcf2d0edc071055aa91d4b0bbae42cef9d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, avg, stddev, sum, count

# Create a Spark session
my_spark = SparkSession.builder.appName("CourseProcessing").getOrCreate()
print (my_spark)

# Data loading and exploration

### Load Coursera Course Dataset

In [ ]:
dataset_path = "/content/coursera_course_dataset.csv"

# Read dataset into DataFrame
df = my_spark.read.csv(dataset_path, header=True, inferSchema=True)

# Explore DataFrame structure and initial rows
df.printSchema()
df.show(5, truncate=False)

root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Organization: string (nullable = true)
 |-- Skills: string (nullable = true)
 |-- Ratings: double (nullable = true)
 |-- Review counts: string (nullable = true)
 |-- Metadata: string (nullable = true)

+---+-------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-----------------+--------------------------------------------------+
|_c0|Title        

#	Data cleaning and preparation

### Extract and Cast Review Count

In [ ]:
# Extract the numeric review count from the "Review counts" column and cast it to an integer
df = df.withColumn(
    "ReviewCount",
    regexp_extract(col("Review counts"), r"\((\d+)K reviews\)", 1))

#cast it to an integer
df=df.withColumn("ReviewCount",df.ReviewCount.cast("integer"))

# Display the extracted review counts
df.select("ReviewCount").show()

+-----------+
|ReviewCount|
+-----------+
|         20|
|        137|
|        100|
|        120|
|         23|
|         73|
|        183|
|         19|
|         75|
|         44|
|         47|
|         42|
|         86|
|         60|
|         47|
|         83|
|         30|
|        142|
|         46|
|         49|
+-----------+
only showing top 20 rows



In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Organization: string (nullable = true)
 |-- Skills: string (nullable = true)
 |-- Ratings: double (nullable = true)
 |-- Review counts: string (nullable = true)
 |-- Metadata: string (nullable = true)
 |-- ReviewCount: integer (nullable = true)



In [ ]:
df.select("Metadata").show()

+--------------------+
|            Metadata|
+--------------------+
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Specia...|
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Profes...|
|Beginner · Specia...|
|Beginner · Specia...|
|Beginner · Profes...|
|Beginner · Specia...|
|Beginner · Specia...|
|Intermediate · Sp...|
|Beginner · Specia...|
|Beginner · Specia...|
+--------------------+
only showing top 20 rows



### Expand Metadata Column

In [ ]:
# Split Metadata column into Level, Certificate_Type, and Duration
df = df.withColumn('Level', split(df['Metadata'], ' · ')[0]) \
       .withColumn('Certificate_Type', split(df['Metadata'], ' · ')[1]) \
       .withColumn('Duration', split(df['Metadata'], ' · ')[2])

# Display expanded columns
df.select("Level", "Certificate_Type", "Duration").show()


+------------+--------------------+------------+
|       Level|    Certificate_Type|    Duration|
+------------+--------------------+------------+
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|      Specialization|1 - 3 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|      Specialization|3 - 6 Months|
|    Beginner|      Specialization|3 - 6 Months|
|    Beginner|Professional Cert...|3 - 6 Months|
|    Beginner|      Specialization|1 - 3 Months|
|    Beginner|      Specialization|3 - 6 Months|
|Intermediate|      

In [ ]:
# Print Dataframe Summary
df.show(5, truncate=False)
df.printSchema()

+---+-------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-----------------+--------------------------------------------------+-----------+--------+------------------------+------------+
|_c0|Title                                |Organization|Skills                                                                                                                                                                                      

### Drop Unnecessary Columns and Review New DataFrame

In [ ]:
new_df = df.drop(*["Metadata_Split", "Review counts", "Metadata"])

# Display and examine the updated DataFrame
new_df.show(5, truncate=False)
new_df.printSchema()

+---+-------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-----------+--------+------------------------+------------+
|_c0|Title                                |Organization|Skills                                                                                                                                                                                                                                                           

#	Descriptive and exploratory analysis

### Analyze Course Titles and Skills

In [ ]:
# Filter courses with high ratings
highly_rated_courses = new_df.filter(col('Ratings') >= 4.5)

print("Courses with Ratings Above 4.5:")
highly_rated_courses.show(10, truncate=False)

Courses with Ratings Above 4.5:
+---+-------------------------------------+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-----------+--------+------------------------+------------+
|_c0|Title                                |Organization      |Skills                                                                                                 

In [ ]:
# Select relevant columns for analysis
title_skills_df = new_df.select(col("Title"), col("Skills"))

# Analyze word frequencies:
#   - Group by Title words and count occurrences
#   - Group by Skill words and count occurrences
#   - Display top 10 frequent words for each

title_word_counts = new_df.groupBy("Title").count().orderBy("count", ascending=False)
print("Top 10 frequent words in course titles:")
title_word_counts.show(10)


skill_word_counts = new_df.groupBy("Skills").count().orderBy("count", ascending=False)
print("Top 10 frequent words in course skills:")
skill_word_counts.show(10)

Top 10 frequent words in course titles:
+--------------------+-----+
|               Title|count|
+--------------------+-----+
|Linear Algebra fo...|    1|
|What is Data Scie...|    1|
|    Entrepreneurship|    1|
|Build a Modern Co...|    1|
|Perform data scie...|    1|
|Python Project fo...|    1|
|Data Science Caps...|    1|
|Customer Service ...|    1|
|Leading Diverse T...|    1|
|     Virtual Reality|    1|
+--------------------+-----+
only showing top 10 rows

Top 10 frequent words in course skills:
+--------------------+-----+
|              Skills|count|
+--------------------+-----+
|     Cloud Computing|    7|
| Cloud Computing,...|    4|
|   Critical Thinking|    3|
|       Communication|    3|
| Communication, L...|    2|
|            Planning|    2|
| HTML and CSS, Re...|    2|
| Leadership and M...|    2|
| Cloud Applicatio...|    2|
| Leadership and M...|    2|
+--------------------+-----+
only showing top 10 rows



### Analyze Course Providers by Average Ratings

In [ ]:
# Select relevant columns
organization_stats_df = new_df.select(col("Organization"), col("Ratings"), col("ReviewCount"), col("Certificate_Type"))

# Group by organization and calculate aggregate statistics
organization_comparison = organization_stats_df.groupBy("Organization").agg(
    avg("Ratings").alias("Average_Ratings"),
    sum("ReviewCount").alias("Total_ReviewCount"),
    count("Certificate_Type").alias("Certificate_Count")
)

# Order the results by average ratings
organization_comparison = organization_comparison.orderBy("Average_Ratings", ascending=False)

# Display the comparison results
organization_comparison.show(truncate=False)

+------------------------------+-----------------+-----------------+-----------------+
|Organization                  |Average_Ratings  |Total_ReviewCount|Certificate_Count|
+------------------------------+-----------------+-----------------+-----------------+
|Akamai Technologies, Inc.     |4.95             |NULL             |2                |
|Arizona State University      |4.9              |31               |3                |
|Hebrew University of Jerusalem|4.9              |NULL             |1                |
|Deep Teaching Solutions       |4.85             |88               |2                |
|Yale University               |4.833333333333333|53               |6                |
|Macquarie University          |4.833333333333333|59               |3                |
|Vanderbilt University         |4.82             |20               |5                |
|Emory University              |4.8              |NULL             |1                |
|Berklee                       |4.8        

### Analyze Correlations between Ratings and ReviewCount

In [ ]:
# Select relevant columns
selected_df = new_df.select(col("Ratings"), col("ReviewCount"))

# Handle missing values
selected_df = selected_df.dropna()

# Calculate and display correlations
correlation_matrix = selected_df.select(corr("Ratings", "ReviewCount"))

# Display the correlation matrix
print("Correlation matrix:")
correlation_matrix.show()

Correlation matrix:
+--------------------------+
|corr(Ratings, ReviewCount)|
+--------------------------+
|        0.2370642575156333|
+--------------------------+



### Analyze Course Duration by Organization and Level

In [ ]:
# Select relevant columns (duration, organization, level) and clean duration data
selected_DOL_df = new_df.select(col("Duration"), col("Organization"), col("Level"))
selected_DOL_df = selected_DOL_df.withColumn("Duration", expr("regexp_replace(Duration, '[^0-9]+', '')").cast("integer"))

# Calculate and display duration statistics
duration_analysis = selected_DOL_df.groupBy("Organization", "Level").agg(
    avg("Duration").alias("Average_Duration")
)

# Show the duration analysis
duration_analysis.show(truncate=False)

+--------------------------------------------------+------------+------------------+
|Organization                                      |Level       |Average_Duration  |
+--------------------------------------------------+------------+------------------+
|Georgia Institute of Technology                   |Intermediate|13.0              |
|University of Minnesota                           |Beginner    |36.0              |
|Northwestern University                           |Beginner    |36.0              |
|Erasmus University Rotterdam                      |Beginner    |13.0              |
|Universidad Nacional Autónoma de México           |Beginner    |13.5              |
|ESSEC Business School                             |Beginner    |36.0              |
|Arizona State University                          |Mixed       |13.0              |
|Microsoft                                         |Intermediate|19.0              |
|Columbia University                               |Mixed       |

### Analyze Course Difficulty Level Distribution

In [ ]:
# Select and group by difficulty level
level_df = new_df.select(col("Level"))

level_distribution = level_df.groupBy("Level").agg(count("*").alias("Course_Count"))

# Display the distribution
level_distribution.show()

+------------+------------+
|       Level|Course_Count|
+------------+------------+
|    Advanced|          18|
|       Mixed|          39|
|Intermediate|         153|
|    Beginner|         413|
+------------+------------+



### Analyze Course Difficulty by Average Ratings and Review Counts

In [ ]:
# Select relevant columns (difficulty level, ratings, review counts)
level_rating_review_df = new_df.select(col("Level"), col("Ratings"), col("ReviewCount"))

# Calculate and display average ratings and total review counts per difficulty
level_analysis = level_rating_review_df.groupBy("Level").agg(
    avg("Ratings").alias("Average_Ratings"),
    sum("ReviewCount").alias("Total_ReviewCount")
)

# Show the analysis results
level_analysis.show()

+------------+-----------------+-----------------+
|       Level|  Average_Ratings|Total_ReviewCount|
+------------+-----------------+-----------------+
|    Advanced|4.627777777777777|             NULL|
|       Mixed|4.633333333333333|               83|
|Intermediate|4.573202614379085|             1063|
|    Beginner|4.667312348668278|             3963|
+------------+-----------------+-----------------+



### Analyze Certificate Type Distribution by Organization

In [ ]:
# Select relevant columns
certificate_org_df = new_df.select(col("Certificate_Type"), col("Organization"))

# Group by certificate type and organization, count occurrences
certificate_analysis = certificate_org_df.groupBy("Certificate_Type", "Organization").agg(
    count("*").alias("Certificate_Count")
)
# Order by certificate count in descending order
certificate_analysis = certificate_analysis.orderBy("Certificate_Count", ascending=False)

# Show the certificate type analysis
certificate_analysis.show(truncate=False)

+------------------------+------------------------------------------+-----------------+
|Certificate_Type        |Organization                              |Certificate_Count|
+------------------------+------------------------------------------+-----------------+
|Course                  |Google                                    |42               |
|Course                  |Google Cloud                              |39               |
|Specialization          |IBM                                       |24               |
|Course                  |IBM                                       |24               |
|Professional Certificate|Google                                    |23               |
|Professional Certificate|IBM                                       |21               |
|Guided Project          |Coursera Project Network                  |19               |
|Specialization          |Google Cloud                              |17               |
|Course                  |DeepLe

### Analyze Total Certificates Issued by Each Organization

In [ ]:
# Calculate and display total certificates per organization (Descending Order)
total_certificates_by_org = certificate_org_df.groupBy("Organization").agg(
    count("*").alias("Total_Certificates_Issued")
)

total_certificates_by_org = total_certificates_by_org.orderBy("Total_Certificates_Issued", ascending=False)

total_certificates_by_org.show(truncate=False)

+------------------------------------------+-------------------------+
|Organization                              |Total_Certificates_Issued|
+------------------------------------------+-------------------------+
|IBM                                       |69                       |
|Google Cloud                              |67                       |
|Google                                    |65                       |
|DeepLearning.AI                           |24                       |
|University of Pennsylvania                |22                       |
|Johns Hopkins University                  |21                       |
|Coursera Project Network                  |19                       |
|Duke University                           |18                       |
|University of Michigan                    |16                       |
|University of Illinois at Urbana-Champaign|15                       |
|University of Colorado Boulder            |14                       |
|Micro